## Adding Party Affiliation to Speakers

In [3]:
import pandas as pd
import os
import re
import requests
import json
from io import StringIO

### Read in processed speeches and state abbreviations

In [5]:
df = pd.read_csv('Results/All_speeches_labelled.csv')
abrv = pd.read_csv("Results/year_names/state_abrv.csv")

### This process was performed manually for each year after iterative key matching

In [1085]:
cong = 102 # Congress to perform matching on

# original subset
oFrame = df.loc[df.congress == cong]

# take relevant data and transform state names to abbreviations
Frame = oFrame.groupby('speaker',as_index=False).first()[['speaker','first_name','last_name','state_x','gender']]
Frame = Frame.merge(abrv[['State','Postal']],left_on='state_x',right_on='State',how='left')


# access ProPublica House Member Database
rq = requests.get(f'https://api.propublica.org/congress/v1/{cong}/house/members.json',
                  headers={"X-API-Key":'WDmcjspeHVFMDSmrIZ3NV2gKAESs5vldAS8rz3X6'})
ref = pd.DataFrame(json.loads(rq.text)['results'][0]['members'])

# extract relevant information and make unique representative identifier
ref = ref[['last_name','first_name','party','state','gender']]
ref['unique_ID'] = ref.apply(lambda x:' '.join([x.first_name,x.last_name,x.state]),1)

# CapitalCase all names to match with record
ref['last_name'] = ref.last_name.str.upper() 
ref['first_name'] = ref.first_name.str.upper()

# Iterative key matching
Name_dict = {}
for i,j in Frame.iterrows():
    sub_ref = ref.loc[ref.last_name == j.last_name]
    if len(sub_ref) > 1: # if more than one match after last name go to state
        sub_ref = sub_ref.loc[sub_ref.state == j.Postal]
        if len(sub_ref) > 1: # if more than one match after state go to gender
            sub_ref = sub_ref.loc[sub_ref.gender == j.gender]
            if len(sub_ref) > 1: # if more than one match after gender go to first name
                sub_ref = sub_ref.loc[sub_ref.first_name == j.first_name] 
    
    # if there are no records, add a blank line for manual coding later
    if len(sub_ref) == 0:
        sub_ref = pd.DataFrame([{"speaker":j.speaker}])
    else:
        sub_ref = sub_ref.reset_index()
        sub_ref['speaker'] = j.speaker
    Name_dict[j.speaker] = sub_ref

# record Keeping and output
longer = []
Results = []
for val in Name_dict.values():
    if len(val) == 1:
        Results.append(val)
    if len(val) > 1:
        longer.append(val)
Result_df = pd.concat(Results)

print("Total Number from Ref - ",len(ref))
print("Single Found - ",len(Result_df.unique_ID.unique()))
print("\nNone Found - ",sum(Result_df.last_name.isnull()))
print('Multiple Found - ',len(longer))

Result_df.to_csv(f'Results/year_names/{cong}_.csv')

100%|██████████| 805/805 [00:23<00:00, 34.80it/s]


original DF length was - 40363
removed 7282 or 0.180413


# Put it all back together

In [1]:
Cleaned = []
for cong in range(98,115):
    sub_df = df.loc[df.congress == cong]
    ref_df = pd.read_csv(f'Results/year_names/{cong}_.csv')
    ref_df = ref_df[['unique_ID','party','gender','speaker']]
    merged = sub_df.merge(ref_df,on='speaker',how='left')
    Cleaned.append(merged)
    
Final = pd.concat(Cleaned)

Final = Final.drop(['Unnamed: 0','Unnamed: 0.1','Unnamed: 0.1.1','state_y',
 'name','party_x', 'chamber_y', 'year_x', 'party_y', 'gender_x', 'int_year'],1)

Final.columns = ['speech_id','speech_text', 'chamber', 'date', 'number_within_file',
 'speaker', 'first_name', 'last_name', 'state', 'line_start', 'line_end', 'file',
 'char_count', 'word_count', 'speech_processed', 'congress', 'window_topic_id',
 'topic_weight', 'year', 'topic_id', 'dynamic_label', 'unique_ID', 'party', 'gender']

Final.to_csv('Results/All_speeches_labelled.csv')